In [1]:
import pandas as pd

In [2]:
way = 'ml-latest-small/'

ratings = pd.read_csv('{}ratings.csv'.format(way))
movies = pd.read_csv('{}movies.csv'.format(way))

### Задание 1

In [3]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [4]:
def getGeo(geo):
    """
    Получение региона из keywords с помощью имен городов этого региона
    """
    
    for k in geo_data:
        if geo in geo_data[k]:
            return k
    
    return 'undefined'

In [5]:
dg = pd.read_csv('keywords.csv')

dg['region'] = dg['keyword'].apply(getGeo)

dg[dg.region != 'undefined'].head()

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток


### Задание 2

In [6]:
def getRating(rat):
    """
    Конверсия рейтинга из среднего балла в систему низкий/средний/высокий рейтинг
    """
    
    if rat <= 2.0:
        return 'низкий рейтинг'
    elif rat <= 4.0:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [7]:
dm = ratings.merge(movies, on='movieId', how='left')

dmg = (dm
       .groupby('movieId')
       .agg({'rating': 'mean'})
       .reset_index()
      )

dmg['class'] = dmg['rating'].apply(getRating)

dm = dm.merge(dmg[['movieId', 'class']], on='movieId', how='left')
dm.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,средний рейтинг
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,средний рейтинг
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,высокий рейтинг


### Задание 3

In [8]:
dr = ratings

drg = (dr.groupby('userId')
       .agg({'timestamp': ['max', 'min'], 'movieId': 'count'})
       .reset_index()
      )

drg = drg[drg['movieId']['count'] >= 100]

drg['Lifetime'] = (drg['timestamp']['max'] - drg['timestamp']['min']) /60 /60 /24
drg.head()

userId   timestamp             movieId    Lifetime
                 max         min   count            
0      1   965719662   964980499     232    8.555127
3      4  1007574542   945078428     216  723.334653
5      6   845556915   845553109     314    0.044051
6      7  1176181731  1106635416     152  804.934201
9     10  1455619275  1455301553     140    3.677338

### Задание 4

In [9]:
years = [y for y in range(1950, 2011)]

In [10]:
def production_year(text_):
    """
    Получение года выпуска фильма из его названия
    """
    
    for y in years:
        if str(y) in text_:
            return y
    
    return 1900

In [11]:
dy = ratings.merge(movies, on='movieId', how='left')

dy['year'] = dy['title'].apply(production_year)

In [12]:
(dy
 .groupby('year')
 .agg({'rating': 'mean'})
 .sort_values('rating', ascending=False)
)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
1964,3.940160
1974,3.935622
1967,3.922572
1975,3.879121
